In [1]:
pip install Mastodon.py

  Using cached Mastodon.py-1.8.1-py2.py3-none-any.whl (65 kB)
  Using cached blurhash-1.1.4-py2.py3-none-any.whl (5.3 kB)
  Using cached python_magic-0.4.27-py2.py3-none-any.whl (13 kB)

[notice] A new release of pip available: 22.1.2 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install pytube

In [2]:
pip install beautifulsoup4


[notice] A new release of pip available: 22.1.2 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import mastodon
from mastodon import Mastodon
from pytube import YouTube
from bs4 import BeautifulSoup
import re
import requests
import urllib


# Log in to Mastodon

In [5]:
# creates an app for mastodon social
Mastodon.create_app(
    'hack-posttoots',
    api_base_url = 'https://mastodon.social',
    scopes=['read', 'write'],
    to_file = '.secrets'
)

('LQ5OWjSAY98wmQj692ThZYVmjsjG8POtQda0YvHcnEM',
 'x3c0ccSrguNKvj_BCiLo9gam49UYTI2BIu7q2tgdsZE')

In [6]:
# saves secrets
mastodon = Mastodon (
    client_id = '.secrets',
)

In [12]:
# requires an existing login

username = '<EMAIL>'
password = '<PASSWORD>'

mastodon.access_token = mastodon.log_in (
  username = username,
  password = password,
  scopes = ['read', 'write']
)
account = mastodon.me()

In [34]:
# some methods for posting and grabbing toot posted on your timeline

# post_toot = mastodon.status_post('Tooting from the command line!')
# toot_status = mastodon.status(post_toot.id)
# mastodon.status_delete(id)
# statuses = mastodon.account_statuses(id=account.id)

# Mastodon parsing

In [21]:
# helpers to parse mastodon payload
def youtube_url_validation(url):
    youtube_regex = (
        r'(https?://)?(www\.)?'
        '(youtube|youtu|youtube-nocookie)\.(com|be)/'
        '(watch\?v=|embed/|v/|.+\?v=)?([^&=%\?]{11})')

    youtube_regex_match = re.match(youtube_regex, url)
    if youtube_regex_match:
        return youtube_regex_match

    return youtube_regex_match


def image_validation(filename):
  image_regex = (r'(.*)(\w+)(.gif|.jpg|.jpeg|.tiff|.png)')
  image_regex_match = re.match(image_regex, filename)
  if image_regex_match:
    return image_regex_match
  return image_regex_match

def video_validation(filename):
  video_regex = (r'(.*)(\w+)\.mp4')
  video_regex_match = re.match(video_regex, filename)
  if video_regex_match:
    return video_regex_match
  return video_regex_match

def grab_video_link(link):
  if video_validation(link):
    return link
  return None

def grab_image_link(link):
  if image_validation(link):
    return link
  return None

def grab_youtube_link(link):
  if youtube_url_validation(link):
    return link
  return None


def grab_links(status_obj):
  image_urls = []
  video_urls = []
  youtube_urls = []
  all_urls = []

  for media_attachment in status_obj['media_attachments']:
    all_urls.append(media_attachment['url'])
    p_image_url = grab_image_link(media_attachment['url'])
    if p_image_url:
      image_urls.append(p_image_url)

    p_video_url = grab_video_link(media_attachment['url'])
    if p_video_url:
      video_urls.append(p_video_url)

  soup = BeautifulSoup(status_obj['content'])
  for a_tag in soup.findAll('a'):
    p_youtube_url = grab_video_link(a_tag.get('href'))
    if p_youtube_url:
      youtube_urls.append(p_youtube_url)
  url_dict = {
      'image_urls': image_urls,
      'video_urls': video_urls,
      'youtue_urls': youtube_urls,
      'all_urls': all_urls 
  }
  return url_dict

In [12]:
# grab media only from
public_statuses = mastodon.timeline_public(only_media=True)

In [ ]:
public_statuses

In [17]:
public_statuses = mastodon.timeline_hashtag('video', limit=100, only_media=True)

In [22]:
for ps in public_statuses:
  print(id, grab_links(ps))

<built-in function id> {'image_urls': ['https://files.mastodon.social/cache/media_attachments/files/110/623/198/254/430/859/original/f4c40bde29923a03.jpg'], 'video_urls': [], 'youtue_urls': [], 'all_urls': ['https://files.mastodon.social/cache/media_attachments/files/110/623/198/254/430/859/original/f4c40bde29923a03.jpg']}
<built-in function id> {'image_urls': ['https://files.mastodon.social/cache/media_attachments/files/110/623/190/286/806/830/original/c8453435a938325f.jpg'], 'video_urls': [], 'youtue_urls': [], 'all_urls': ['https://files.mastodon.social/cache/media_attachments/files/110/623/190/286/806/830/original/c8453435a938325f.jpg']}
<built-in function id> {'image_urls': ['https://files.mastodon.social/cache/media_attachments/files/110/622/965/509/611/727/original/3b605550a80a5473.jpg'], 'video_urls': [], 'youtue_urls': [], 'all_urls': ['https://files.mastodon.social/cache/media_attachments/files/110/622/965/509/611/727/original/3b605550a80a5473.jpg']}
<built-in function id> {'